In [10]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

In [11]:
def get_players(league,year):
    url = 'https://understat.com/league/' + league + '/' + year #EPL/2023'
    res = requests.get(url)
    soup = BeautifulSoup(res.content,'lxml')
    scripts = soup.find_all('script')
    strings = scripts[3].string
    ind_start = strings.index("('")+2
    ind_end = strings.index("')")
    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)
    return data

In [12]:
def scrap(data):
    players = []
    xg = []
    xa = []
    npxg = []
    npxga = []
    position = []
    goals = []
    npgoals = []
    assists = []
    shots = []
    npg = []
    time = []
    
    for index in data:
        players.append(index['player_name'])
        xg.append(index['xG'])
        xa.append(index['xA'])
        shots.append(index['shots'])
        npxg.append(index['npxG'])
        npg.append(index['npg'])
        position.append(index['position'])
        goals.append(index['goals'])
        assists.append(index['assists'])
        time.append(index['time'])
        
    col_name = ['player_name','goals','assists','shots','npg','xg','npxg','xa','position','time']
    df = pd.DataFrame([players,goals,assists,shots,npg,xg,npxg,xa,position,time],index=col_name)
    df = df.T
    return df

In [13]:
def format_all(df2):
    df2.goals = df2.goals.astype(float)
    df2.assists = df2.assists.astype(float)
    df2.shots = df2.shots.astype(float)
    df2.npg = df2.npg.astype(float)
    df2.xg = df2.xg.astype(float)
    df2.npxg = df2.npxg.astype(float)
    df2.xa = df2.xa.astype(float)
    df2.time = df2.time.astype(float)
    
    df2.xg = df2.xg.apply(lambda x: round(x, 2))
    df2.npxg = df2.npxg.apply(lambda x: round(x, 2))
    df2.xa = df2.xa.apply(lambda x: round(x, 2))
    
    df2 = df2[df2['time'] > 500]
    df2 = df2[(df2['goals'] > 1) & (df2['assists'] > 0)].sort_values(by = "goals", ascending = False)
    return df2

In [14]:
def main():
    data = get_players('EPL','2023')
    df = scrap(data)
    df = format_all(df)
    path =  r'C:/Users/jakub/Documents/kursy/python/python_understat/understat_players_all.xlsx'
    with pd.ExcelWriter(path) as writer:
        df.to_excel(writer, sheet_name="all_strikers")

In [15]:
main_all()